# Collect all nattyorjuice post urls to manually gather data

## Install/import stuff

In [ ]:
#hide
!pip install .[dev]

#!jupyter nbextension install https://github.com/drillan/jupyter-black/archive/master.zip --user
#!jupyter nbextension enable jupyter-black-master/jupyter-black

!nbdime extensions --enable --user

Defaulting to user installation because normal site-packages is not writeable
Processing /home/cedar/source/steroids-or-not
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.


  Created wheel for steroids-or-not: filename=steroids_or_not-0.0.1-py3-none-any.whl size=4107 sha256=4b5f56597828bcf53ba1f21a71c552ce42c67d032632cf271273096f3caa106e
  Stored in directory: /home/cedar/.cache/pip/wheels/8b/e8/f0/812780b69e692fdae0be65743cbacdc466f0d0c0cc3446797e
Successfully built steroids-or-not
  Attempting uninstall: steroids-or-not
    Found existing installation: steroids-or-not 0.0.1
    Uninstalling steroids-or-not-0.0.1:
      Successfully uninstalled steroids-or-not-0.0.1
You should consider upgrading via the '/usr/bin/python3 -m pip install --upgrade pip' command.
Enabling: nbdime
- Writing config: /home/cedar/.jupyter
    - Validating...
      nbdime 3.1.0 OK
Installing /home/cedar/.local/lib/python3.6/site-packages/nbdime/notebook_ext -> nbdime
Up to date: /home/cedar/.local/share/jupyter/nbextensions/nbdime/nbdime.yaml
Up to date: /home/cedar/.local/share/jupyter/nbextensions/nbdime/index.js
- Validating: OK

    To initialize this nbextension in the brows

In [ ]:
# hide

## Loads environment variables from .env file

%load_ext dotenv
%dotenv
import os
import requests
import time
import datetime
import sys
import pickle
from requests.adapters import HTTPAdapter
from collections import Counter
import webbrowser

## Data discovery

In [ ]:
pushshift = "https://api.pushshift.io/reddit/"
pushshift_submission_url = pushshift + "submission/search"
one_year_seconds = 1 * 365 * 24 * 60 * 60
before = int(time.time() - (one_year_seconds * 9))

params = {
    "subreddit": "nattyorjuice",
    "size": "25",
}

In [ ]:
# t0 = time.time()
response = requests.get(pushshift_submission_url, params=params)
# t1 = time.time()

# total = t1-t0

In [ ]:
# total

In [ ]:
response

<Response [200]>

In [ ]:
len(response.json()['data'])

25

In [ ]:
response.json()['data'][0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'NXGZ',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_7k499ptf',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1628550257,
 'crosspost_parent': 't3_p1c3c8',
 'crosspost_parent_list': [{'all_awardings': [],
   'allow_live_comments': False,
   'approved_at_utc': None,
   'approved_by': None,
   'archived': False,
   'author': 'SamMee514',
   'author_flair_background_color': None,
   'author_flair_css_class': None,
   'author_flair_richtext': [],
   'author_flair_template_id': None,
   'author_flair_text': None,
   'author_flair_text_color': None,
   'author_flair_type': 'text',
   'author_fullname': 't2_6eyqt',
   'author_is_blocked': False,
   'author_patreon_flair': False,
   'author_premium': True,
   'aw

In [ ]:
response.json()['data'][0]

{'all_awardings': [],
 'allow_live_comments': False,
 'author': 'NXGZ',
 'author_flair_css_class': None,
 'author_flair_richtext': [],
 'author_flair_text': None,
 'author_flair_type': 'text',
 'author_fullname': 't2_7k499ptf',
 'author_is_blocked': False,
 'author_patreon_flair': False,
 'author_premium': False,
 'awarders': [],
 'can_mod_post': False,
 'contest_mode': False,
 'created_utc': 1628550257,
 'crosspost_parent': 't3_p1c3c8',
 'crosspost_parent_list': [{'all_awardings': [],
   'allow_live_comments': False,
   'approved_at_utc': None,
   'approved_by': None,
   'archived': False,
   'author': 'SamMee514',
   'author_flair_background_color': None,
   'author_flair_css_class': None,
   'author_flair_richtext': [],
   'author_flair_template_id': None,
   'author_flair_text': None,
   'author_flair_text_color': None,
   'author_flair_type': 'text',
   'author_fullname': 't2_6eyqt',
   'author_is_blocked': False,
   'author_patreon_flair': False,
   'author_premium': True,
   'aw

In [ ]:
response.json()['data'][0]['permalink']

'/r/nattyorjuice/comments/p1cr65/this_70_years_old_man_looks_like_30s_after_34/'

In [ ]:
response.json()['data'][0]['full_link']

'https://www.reddit.com/r/nattyorjuice/comments/p1cr65/this_70_years_old_man_looks_like_30s_after_34/'

In [ ]:
response.json()['data'][0]['url']

'https://streamable.com/gjsoc5'

## Crawling/ Downloading Data from pushshift

In [ ]:
class SubredditCrawler(object):
    """
    Borrowed heavily from here: https://www.textjuicer.com/2019/07/crawling-all-submissions-from-a-subreddit/
    """

    pushshift = "https://api.pushshift.io/reddit/"
    pushshift_submission_url = pushshift + "submission/search"

    def __init__(self, subreddit, file_path, max_submissions=200):
        self.subreddit = subreddit
        self.file_path = file_path
        self.max_submissions = max_submissions
        self.submissions = []

        self.session = requests.Session()
        self.session.mount('http://api.pushshift.io/', HTTPAdapter(max_retries=5))

In [ ]:
class SubredditCrawler(SubredditCrawler):
    def crawl_page(self, last_page=None):
        """
        Crawl a page of results from a given subreddit.

        :param subreddit: The subreddit to crawl.
        :param last_page: The last downloaded page.

        :return: A page or results.
        """
        params = {
            "subreddit": self.subreddit,
            "size": 100,
            "sort": "desc",
            "sort_type": "created_utc",
        }
        if last_page is not None:
            if len(last_page) > 0:
                # resume from where we left at the last page
                params["before"] = last_page[-1]["created_utc"]
            else:
                # the last page was empty, we are past the last page
                return []
        results = self.session.get(pushshift_submission_url, params=params)
        
        if not results.ok:
            # something wrong happened
            raise Exception(
                "Server returned status code {}".format(results.status_code)
            )
        return results.json()["data"]

In [ ]:
class SubredditCrawler(SubredditCrawler):
    def crawl_subreddit(self, erase_self_submissions=False):
        """
        Crawl submissions from a subreddit.
        Isn't strictly correct on the number of submissions it grabs but doesn't matter too much
        for our purposes

        :param subreddit: The subreddit to crawl.
        :param max_submissions: The maximum number of submissions to download.

        :return: A list of submissions.
        """
        last_page = None

        if erase_self_submissions:
            self.submissions = []

        print(f"Started {datetime.datetime.now()}")

        with open(self.file_path, "wb") as file:

            while last_page != [] and len(self.submissions) < self.max_submissions:
                last_page = self.crawl_page(last_page)

                self.submissions += last_page
                pickle.dump(last_page, file)
                print(f"---- pickled {len(self.submissions)} posts so far ------")
                print(f"Last post title: {last_page[-1]['title']}")

                time.sleep(3)

        print(f"--------------------------------------")
        print(f"Finished {datetime.datetime.now()}")
        print(f"PICKLED {len(self.submissions)} SUBMISSIONS to file:")
        print(f"{self.file_path}")
        print(f"--------------------------------------")

In [ ]:
def open_list(urls):
    '''
    Opens a list of urls in web browser for convenience in viewing data
    '''
    [webbrowser.open(url) for url in urls]

NameError: name 'crawler' is not defined

## Unpickling, cleaning downloaded pushshift data

In [ ]:
pickled_posts = []
with open('data/all_nattyorjuice_submissions.pkl', "rb") as file:
    while 1:
        try:
            pickled_posts += pickle.load(file)
        except EOFError:
            break

len(pickled_posts)

41725

### Remove invalid posts

In [ ]:
mostly_valid = [post for post in pickled_posts if post['num_comments'] >= 2]
len(mostly_valid)

34356

### Most common post link domains

In [ ]:
Counter([post['domain'] for post in mostly_valid]).most_common()

[('i.redd.it', 17502),
 ('instagram.com', 3932),
 ('self.nattyorjuice', 3762),
 ('reddit.com', 2248),
 ('i.imgur.com', 1610),
 ('imgur.com', 1340),
 ('youtube.com', 777),
 ('youtu.be', 490),
 ('v.redd.it', 290),
 ('i.reddituploads.com', 105),
 ('facebook.com', 84),
 ('m.imgur.com', 53),
 ('pbs.twimg.com', 47),
 ('i.pinimg.com', 46),
 ('m.youtube.com', 45),
 ('twitter.com', 43),
 ('gfycat.com', 39),
 ('i.ytimg.com', 37),
 ('vm.tiktok.com', 20),
 ('streamable.com', 17),
 ('google.com', 16),
 ('self.Fitness', 15),
 ('dailymail.co.uk', 15),
 ('i.4cdn.org', 15),
 ('s-media-cache-ak0.pinimg.com', 15),
 ('images.app.goo.gl', 13),
 ('ibb.co', 13),
 ('i.dailymail.co.uk', 13),
 ('external-preview.redd.it', 11),
 ('i.gyazo.com', 10),
 ('girlswithmuscle.com', 10),
 ('np.reddit.com', 10),
 ('i.redditmedia.com', 10),
 ('scontent.cdninstagram.com', 10),
 ('preview.redd.it', 9),
 ('old.reddit.com', 9),
 ('tiktok.com', 8),
 ('encrypted-tbn0.gstatic.com', 8),
 ('i.instagram.com', 7),
 ('yt3.ggpht.com', 

### i.redd.it posts AKA just the images

In [ ]:
i_redd_it = [post for post in mostly_valid if post['domain'] == 'i.redd.it']
len(i_redd_it)

17502

### Remove certain flair e.g. Meme

In [ ]:
flaired = [post for post in i_redd_it if 'link_flair_text' in post]
len(flaired)

5247

In [ ]:
Counter([post['link_flair_text'] for post in flaired]).most_common()

[('Natty or Juice?', 3732),
 ('Meme', 455),
 ('Discussion', 372),
 ('Tough Question', 312),
 ('GirlPower', 152),
 ('Feat of Strength', 113),
 ('Snu_Snu', 40),
 ('JUICY', 27),
 ('CALVINS', 18),
 ('FAKE NATTY', 18),
 ('NATTY', 3),
 ('PHOTOSHOP', 1),
 ('VERY JUICY', 1),
 ('JUICE V FAKENATTY', 1),
 ('BOTH JUICY', 1),
 ('SHITPOST', 1)]

In [ ]:
def exclude_irrelevant_posts(posts):
    relevant_posts = []
    for post in posts:
        if 'link_flair_text' in post:
            flair = post['link_flair_text']
            if flair == 'Meme':
                # Memes are not useful for our purposes, exclude them
                continue
        # Include everything else
        relevant_posts.append(post)
    return relevant_posts

In [ ]:
pictures_no_memes = exclude_irrelevant_posts(i_redd_it)
len(pictures_no_memes)

17047

### Investigate how to exclude deleted posts

In [ ]:
response.json()